In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import os
import numpy as np
import pandas as pd
import cv2
from tqdm import tqdm
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Input
from tensorflow.keras.models import Model
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.optimizers import Adam


In [ ]:
DATASET_DIR = "/kaggle/input/utkface-new"
UTKFACE_DIR = os.path.join(DATASET_DIR, "UTKFace")
CROP_DIR = os.path.join(DATASET_DIR, "crop_part1")

IMG_SIZE = 224


In [ ]:
def load_image_paths(folder):
    paths = []
    for file in os.listdir(folder):
        if file.endswith(".jpg"):
            paths.append(os.path.join(folder, file))
    return paths

image_paths = load_image_paths(UTKFACE_DIR) + load_image_paths(CROP_DIR)

print("Total images found:", len(image_paths))


In [ ]:
def parse_labels(path):
    try:
        filename = os.path.basename(path)
        age, gender, _, _ = filename.split("_", 3)
        age = int(age)
        gender = int(gender)
        if age < 0 or age > 100:
            return None
        return age, gender
    except:
        return None


In [ ]:
data = []

for path in image_paths:
    labels = parse_labels(path)
    if labels:
        data.append([path, labels[0], labels[1]])

df = pd.DataFrame(data, columns=["path", "age", "gender"])


In [ ]:
from sklearn.model_selection import train_test_split

train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)


In [ ]:
IMG_SIZE = 224
BATCH_SIZE = 32

def load_image(path, age, gender):
    img = tf.io.read_file(path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, (IMG_SIZE, IMG_SIZE))
    img = img / 255.0
    return img, {"gender": gender, "age": age}


In [ ]:
def make_dataset(df, training=True):
    ds = tf.data.Dataset.from_tensor_slices(
        (df["path"].values, df["age"].values, df["gender"].values)
    )

    ds = ds.map(load_image, num_parallel_calls=tf.data.AUTOTUNE)

    if training:
        ds = ds.shuffle(1024)

    ds = ds.batch(BATCH_SIZE)
    ds = ds.prefetch(tf.data.AUTOTUNE)
    return ds


In [ ]:
train_ds = make_dataset(train_df, training=True)
val_ds = make_dataset(val_df, training=False)


In [ ]:
base_model = ResNet50(
    weights="imagenet",
    include_top=False,
    input_shape=(IMG_SIZE, IMG_SIZE, 3)
)

for layer in base_model.layers:
    layer.trainable = False


In [ ]:
from tensorflow.keras.layers import Dropout
x = GlobalAveragePooling2D()(base_model.output)
x = Dense(256, activation="relu")(x)
x=Dropout(0,3)(x)

gender_output = Dense(1, activation="sigmoid", name="gender")(x)
age_output = Dense(1, activation="linear", name="age")(x)

model = Model(
    inputs=base_model.input,
    outputs=[gender_output, age_output]
)


In [ ]:
model.compile(
    optimizer=Adam(learning_rate=1e-3),
    loss={
        "gender": "binary_crossentropy",
        "age": "mse"
    },
    metrics={
        "gender": "accuracy",
        "age": "mae"
    }
)



In [ ]:
train_ds = make_dataset(train_df, training=True)
val_ds = make_dataset(val_df, training=False)


In [ ]:
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10
)


In [ ]:
for layer in base_model.layers[-50:]:
    layer.trainable = True

model.compile(
    optimizer=Adam(learning_rate=1e-4),
    loss={
        "gender": "binary_crossentropy",
        "age": "mse"
    },
    metrics={
        "gender": "accuracy",
        "age": "mae"
    }
)


In [ ]:
model.evaluate(val_ds)


In [ ]:
model.save("age_gender_model.h5")


In [ ]:
def predict_image(path):
    img = tf.io.read_file(path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, (224,224))
    img = img / 255.0
    img = tf.expand_dims(img, axis=0)

    gender_pred, age_pred = model.predict(img)
    gender = "Male" if gender_pred[0][0] < 0.5 else "Female"
    age = int(age_pred[0][0])
    return gender, age
